# ELT Project

## Finding Data:

DATA SOURCE: https://www.kaggle.com/datasnaek/youtube-new/data <br/>
Utilizing: <br/>
3 csv files with Video Information (Canada, US, and Britain) <br/>
3 json files with Category Assignment (Canada, US, and Britain) <br/>

## Data Cleanup & Analysis

Plan and document the following:
* The sources of data that you will extract from.
* The type of transformation needed for this data (cleaning, joining, filtering, aggregating, etc).
* The type of final production database to load the data into (relational or non-relational).
* The final tables or collections that will be used in the production database.

You will be required to submit a final technical report with the above information and steps required to reproduce your ETL process.

## Project Report:

Submit a Final Report that describes the following:
* Extract: your original data sources and how the data was formatted (CSV, JSON, pgAdmin 4, etc).
* Transform: what data cleaning or transformation was required.
* Load: the final database, tables/collections, and why this was chosen.

Please upload the report to Github and submit a link to Bootcampspot.

In [1]:
import os
import pandas as pd
import json
import requests

from pandas.io.json import json_normalize
from sqlalchemy import create_engine

# EXTRACT

In [2]:
json_CA = os.path.join("data", "CA_category_id.json")
category_CA_df = pd.read_json(json_CA)
CA_df = json_normalize(category_CA_df['items'])

In [3]:
json_US = os.path.join("data", "US_category_id.json")
category_US_df = pd.read_json(json_US)
US_df = json_normalize(category_US_df['items'])

In [4]:
json_MX = os.path.join("data", "GB_category_id.json")
category_GB_df = pd.read_json(json_MX)
GB_df = json_normalize(category_GB_df['items'])

In [5]:
csv_GB = os.path.join("data", "GBvideos.csv")
GB_csv_df = pd.read_csv(csv_GB)

In [6]:
csv_US = os.path.join("data", "USvideos.csv")
US_csv_df = pd.read_csv(csv_US)

In [7]:
csv_CA = os.path.join("data", "CAvideos.csv")
CA_csv_df = pd.read_csv(csv_CA)

# TRANSFORM

In [8]:
US_df.head(1)

,etag,id,kind,snippet.assignable,snippet.channelId,snippet.title
0,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,youtube#videoCategory,True,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation


In [9]:
US_csv_df.head(1)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...


# LOAD

FULL DATAFRAME +
* top 5 per country
* average rank by category overall
* average rank by category by country
* number of videos per category
* number of videos per category by country
* average number of views of top 10 overall
* average number of views of top 10 by country
* average number of view overall by country